In [2]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json

# !conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim

from bs4 import BeautifulSoup

import requests
from pandas import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

import folium

print('Libraries imported.')

Libraries imported.


Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas  dataframe.

The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

In [26]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
source = requests.get(url)
soup = BeautifulSoup(source.text, 'lxml')

data = []
coloumns = []
table = soup.find('table')

for index, tr in enumerate(table.find_all('tr')):
    section = []
    
    for td in tr.find_all(['th', 'td']):
        section.append(td.text.rstrip())
        
    
    if index == 0:
        columns = section
    else:
        data.append(section)
    
    
          
df_post_code = pd.DataFrame(data = data, columns = columns)
print(df_post_code.shape)
df_post_code.head()

(180, 3)


,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [27]:
df_post_code = df_post_code[df_post_code['Borough'] != 'Not assigned']

More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11  in the above table.

In [28]:
df_post_code = df_post_code.groupby(['Postal Code', 'Borough'])['Neighbourhood'].apply(','.join).reset_index()
df_post_code.drop_duplicates(inplace=True)

df_post_code.set_index('Postal Code', inplace=True)
df_post_code.head()

,Borough,Neighbourhood
Postal Code,,
M1B,Scarborough,"Malvern, Rouge"
M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
M1E,Scarborough,"Guildwood, Morningside, West Hill"
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae


In [29]:
df_post_code.loc[df_post_code['Neighbourhood'] == 'Not assigned', 'Neighbourhood'] = df_post_code.loc[df_post_code['Neighbourhood'] == 'Not assigned', 'Borough']
df_post_code.head()

,Borough,Neighbourhood
Postal Code,,
M1B,Scarborough,"Malvern, Rouge"
M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
M1E,Scarborough,"Guildwood, Morningside, West Hill"
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae


In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [31]:
df_post_code.shape

(103, 2)